# __PRACTICUM PROJECT CODE__

## PART 1: *Import libraries and configure settings*

In [ ]:
#Import basic libraries
import pandas as pd
import numpy as np
import datetime
import gspread
from datetime import timedelta
from collections import defaultdict

#Install Machine Learning Packages
#!pip install sklearn
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import SMOTENC

#Import packages for oversampling and undersampling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling._smote.base import Counter
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## PART 2: *Read data and clean*

In [ ]:
#Read in the change dataset
change_df = pd.read_excel('Datasets/Changes_Clean.xlsx')
#Read in the incident data set
incident_df = pd.read_excel('Datasets/Incidents_Clean.xlsx')

In [ ]:
#Display first five rows
change_df.head(5)

In [ ]:
#Data Cleaning on Change Dataset

##Convert u_total_change_window to int
change_df['u_total_change_window'] = change_df['u_total_change_window'].astype(int)

##Convert date columns to datetime format. 
date_cols =['start_date','opened_at','closed_at']
for x in date_cols:
    change_df[x] = pd.to_datetime(change_df[x])
    change_df[x] = change_df[x].fillna(pd.Timestamp("20500101 00:00:00")) #Fill empty dates with dummy date

##Fill empty cells with 'Unknown or Not Applicable'
##Columns to fill
string_cols = ['u_reason_for_emergency','u_applications_impacted.1_map','u_applications_impacted.2_map','u_applications_impacted.3_map','u_applications_impacted.4_map','u_applications_impacted.5_map',
               'u_business_map','assignment_group_map','state','risk','u_platform_impacted.1_map','u_platform_impacted.2_map','u_platform_impacted.3_map','u_platform_impacted.4_map','u_platform_impacted.5_map',
               'u_application_criticality','u_environment','u_inc_prob_ref','u_validation_status']

for x in string_cols:
    change_df[x] = change_df[x].fillna("Unknown or Not Applicable")

#Add a column for the hour of start of change
change_df['hour_start'] = change_df['start_date'].dt.hour

In [ ]:
#Data Cleaning on Incidents Dataset

##Columns to change
inc_date_cols =['resolved_at','opened_at','closed_at']

##Convert date columns to datetime format
for x in inc_date_cols:
    incident_df[x] = pd.to_datetime(incident_df[x])
    incident_df[x] = incident_df[x].fillna(pd.Timestamp("20500101 00:00:00")) #Fill empty values with dummy dates
    

##Fill empty cells with 'Unknown or Not Applicable'
##Columns to fill
inc_str_cols =['number',
'priority',
'state',
'category',
'assignment_group_map',
'u_application_map',
'u_business_map',
'caused_by_change',
'impact',
'u_platform_map',
'root_cause',
'u_task_categorization',
'urgency',
'u_type',
'subcategory',
'cmdb_ci_map',
'close_code',
'u_environment',
'u_app_criticality',
'u_users_affected',
'u_service_impact',
'u_customer_impact',
'u_recovery_complexity',
'u_resource_availability',
'u_increased_impact_time',
'u_expected_time_to_recover']

for x in inc_str_cols:
    incident_df[x] = incident_df[x].fillna("Unknown or Not Applicable")
    
##Replace values in the 'u_environment' column of the DataFrame incident_df to standardize names
incident_df['u_environment'] = incident_df['u_environment'] \
                               .replace(['Prod','User Acceptance Testing (UAT)'],['Production','UAT'])

In [ ]:
#Merge the change with the incident dataset on 'caused_by_change' columm
merged_df = change_df.merge(incident_df,how='inner',left_on='number',right_on='caused_by_change')

In [ ]:
#New column to store the hour at which the change was started
merged_df['start_hour']  = merged_df['start_date'].dt.hour

#Of the changes that caused incidents, what hour of the day were they started?
#merged_df.groupby('start_hour')[['start_hour']].agg(['count']) 

## PART 3: *Feature engineering*

In this section, we create new features for the incident_df dataframe to use in subsequent machine learning models. Eight new columns are created in total. The table below displays the new columns and their purpose.

| New Feature | Explanation |
| :- | :- |
| **risk_rating**   | Tracks riskiness of application based on **changes** |
| **app_risk_rating**   | Tracks riskiness of application based on **incidents**  |
| **inc_app_counts**   | Calculates the 80th, 90th, 95th, and 97.5th percentiles of the values in app_counts |
| **percent_caused_by_change**   | Calculates percentage of incidents in each application that were caused by a change |
| **bus_risk_rating**   | Tracks riskiness of application based on the business units affected by change  |
| **inc_business_counts**   | Calculates the 80th, 90th, 95th, and 97.5th percentiles of the values in inc_business_counts |
| **group_risk_rating**   | Tracks the riskiness of the group that the change is assigned to |
| **assignment_counts**  | Calculates the 90th, 90th, 95th, and 97.5th percentiles of the values in assignment_counts |

Four columns are created are created by mapping values into categories, ranging from low to high. These represent the riskiness of the application, group, or business unit based on the number of incidents or changes involved. The other columns serve two primary purposes: (a) They aid in bucketing numerical values into the relevant categories, and/or (b) They provide a sense of which applications, groups, or business units might be particularly problematic.

In [ ]:
#FEATURE ENGINEERING


#ADD ONE NEW COLUMN (risk_rating) TO incident_df

#A dataframe to hold Changes, along with the ICE applications impacted by the change
app_df = merged_df[['number_x','u_applications_impacted.1_map','u_applications_impacted.2_map',
                    'u_applications_impacted.3_map','u_applications_impacted.4_map',
                    'u_applications_impacted.5_map']]
#Melt the u_applications_impacted.X_map columns, turning them into rows
new_app = pd.melt(app_df, 
                  id_vars =['number_x'], 
                  value_vars =['u_applications_impacted.1_map','u_applications_impacted.2_map',
                               'u_applications_impacted.3_map','u_applications_impacted.4_map',
                               'u_applications_impacted.5_map'])
#Drop unneeded duplicate rows
new_app = new_app.drop_duplicates()
#Create a new column to assign a risk rating to each application.
##If the application was affected a certain number of times, rate accordingly
app_counts = new_app.groupby('value') \
                .agg(app_counts=('value','count')) \
                .sort_values(by='app_counts',ascending=False) \
                .reset_index()
def mapping(value):
    if value < 11.4:
        return'Low'
    elif value >= 11.4 and value < 20:
        return 'Medium'
    elif value >= 20 and value < 29:
        return 'Medium High'
    elif value >= 29:
        return 'High'
app_counts['risk_rating'] = app_counts['app_counts'].map(mapping)
#Add the counts and risk rating to the incidents data frame
incident_df = incident_df.merge(app_counts,how='left',left_on='u_application_map',right_on='value')
#Fill empty values of the risk_rating column
incident_df['risk_rating'] = incident_df['risk_rating'].fillna("Unknown or Not Applicable")



#ADD TWO NEW COLUMNS (app_risk_rating and inc_app_counts_df) to incident_df

#Which applications are associated with the most incidents?
inc_app_counts_df = merged_df.groupby('u_application_map') \
                            .agg(inc_app_counts=('u_application_map','count')) \
                            .sort_values(by='inc_app_counts',ascending=False) \
                            .reset_index()
#Create a new column to assign a risk rating to each application.
##If the application was affected a certain number of times, rate accordingly
def inc_app_mapping(value):
    if value < 5:
        return'Low'
    elif value >= 5 and value < 9:
        return 'Medium'
    elif value >= 9 and value <13:
        return 'Medium High'
    elif value >= 13:
        return 'High'

inc_app_counts_df['app_risk_rating'] =  inc_app_counts_df['inc_app_counts'].map(inc_app_mapping)
inc_app_counts_df['inc_app_counts'].quantile([.80,.90,.95,.975])
incident_df = incident_df.merge(inc_app_counts_df,how='left',left_on='u_application_map',right_on='u_application_map')



#ADD ONE NEW COLUMN (inc_percentage) to incident_df.
#This column tracks the percent of incidents in each application that were caused by a change
all_inc_counts = incident_df.groupby('u_application_map') \
                  .agg(inc_app_counts=('u_application_map','count')) \
                  .sort_values(by='inc_app_counts',ascending=False) \
                  .reset_index()
inc_percentage = all_inc_counts.merge(inc_app_counts_df,how='inner', on='u_application_map')
inc_percentage['percent_caused_by_change']  = inc_percentage.inc_app_counts_y/inc_percentage.inc_app_counts_x
inc_percentage = inc_percentage[(inc_percentage['inc_app_counts_x'] != inc_percentage['inc_app_counts_y']) 
                                & (inc_percentage['inc_app_counts_x'] > 5)
                                & (inc_percentage['inc_app_counts_y'] > 3)]
incident_df = incident_df.merge(inc_percentage,how='left',left_on='u_application_map',right_on='u_application_map')



#ADD TWO NEW COLUMNS (bus_risk_rating and inc_business_counts) to incident_df
#Create a new column to assign risk rating for each business unit
inc_business_unit = merged_df.groupby('u_business_map_y') \
                    .agg(inc_business_counts=('u_business_map_y','count')) \
                    .sort_values(by='inc_business_counts',ascending=False) \
                    .reset_index()
def bus_unit_mapping(value):
    if value < 88:
        return'Low'
    elif value >= 88 and value < 155:
        return 'Medium'
    elif value >= 155 and value <184:
        return 'Medium High'
    elif value >= 247.5:
        return 'High'

inc_business_unit['bus_risk_rating'] =  inc_business_unit['inc_business_counts'].map(bus_unit_mapping)
inc_business_unit['inc_business_counts'].quantile([.80,.90,.95,.975])
incident_df = incident_df.merge(inc_business_unit,how='left',left_on='u_business_map',right_on='u_business_map_y')


#ADD TWO NEW COLUMNS (assignment_counts and group_risk_rating) to incident_df
#Create a new column to assign a risk rating to each application affected by a change
assignment_group_df = merged_df.groupby('assignment_group_map_y') \
                        .agg(assignment_counts=('assignment_group_map_y','count')) \
                        .sort_values(by='assignment_counts',ascending=False) \
                        .reset_index()
def assign_group_mapping(value):
    if value < 191.4:
        return'Low'
    elif value >= 191.4 and value <288.6 :
        return 'Medium'
    elif value >= 288.6 and value <375.35 :
        return 'Medium High'
    elif value >= 375.35:
        return 'High'
    
assignment_group_df['assignment_counts'].quantile([.80,.90,.95,.975])
assignment_group_df['group_risk_rating'] =  assignment_group_df['assignment_counts'].map(bus_unit_mapping)
incident_df = incident_df.merge(assignment_group_df,how='left',left_on='assignment_group_map',right_on='assignment_group_map_y')


In [ ]:
#Show final version of incident_df
incident_df.head(10)

## PART 4: *Machine learning for Task 1*

In this section, we construct and test a machine learning model to classify incidents according to the changes that caused them. This task is divided into two parts: **Task 1a** and **Task 1b**. 

In **Task 1a**, we define a binary classification model to determine whether or not an incident was caused by a change. Oversampling and undersampling techniques are employed to handle class imbalancing in the dataset. 5-fold cross validation is employed to estimate model accuracy. Several models are tested and compared. (Note to team: We can list these other models here once we decide which to use.) It is found that Random Forest Classifier performs best overall. (Note to team: We can compare the results in a table here.)

In **Task 1b**, we construct an algorithm to determine the probability that an incident is caused by a particular change. (Note to team: I will put more detail here this week.)

## **Task 1a** ##

In [ ]:
#Flag incidents in the DataFrame that have a valid CMDB configuration item (CI) associated with them 
#(i.e., "cmdb_ci_map" is not "Unknown or Not Applicable"). 
#These incidents will be flagged with a value of 1 in the "cmdb_flag" column
#Incidents without a valid CI will be flagged with a value of 0.
incident_df['cmdb_flag'] = incident_df.cmdb_ci_map.map(lambda x: 0 if x =='Unknown or Not Applicable' else 1)

#Take columns for machine learning - part 1
inc_ml = incident_df[['root_cause', 'impact','u_business_map','u_task_categorization','app_risk_rating_y',
                    'u_customer_impact','u_recovery_complexity','bus_risk_rating',
                    'group_risk_rating','percent_caused_by_change','caused_by_change']]

#Prepare features for machine learning by handling replacing null values
inc_ml['percent_caused_by_change'] = inc_ml['percent_caused_by_change'].fillna(0)
inc_ml['group_risk_rating'] = inc_ml['group_risk_rating'].fillna('No risk')
inc_ml['bus_risk_rating'] = inc_ml['bus_risk_rating'].fillna('No risk')

#Create the response binary response variable
inc_ml.loc[:,'response'] = inc_ml.caused_by_change.map(lambda x: 0 if x =='Unknown or Not Applicable' else 1)

In [ ]:
#Show dataframe
inc_ml.head(5)

In [ ]:
#Perform oversampling and undersampling to handle class imbalancing in the dataset

#Gather predictors (X) and response variable (y)
X = inc_ml.iloc[:,0:10]
y = inc_ml.iloc[:,-1]

#Gather categorical features into list
cat_vars = ['root_cause', 'impact','u_business_map','u_task_categorization','app_risk_rating_y',
                    'u_customer_impact','u_recovery_complexity','bus_risk_rating','group_risk_rating']

#Convert the categorical variables into numerical labels, allowing them to be used as input to machine learning models.
label_encoder = LabelEncoder()
for i in cat_vars:
    X[i] = label_encoder.fit_transform(X[i])

#Split data into training and testing data sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=42, shuffle=True)

#Oversampling - Generate synthetic data points to balance the class distribution of a dataset.
#The minority class will be increased to 10% of the majority class.
over = SMOTENC(sampling_strategy=.1,categorical_features=[0,1,2,3,4,5,6,7,8])

#Undersampling - randomly remove some of the majority class samples to further balance the class distribution of the dataset.
#The majority class will be reduced to 50% of the minority class.
under = RandomUnderSampler(sampling_strategy=.5)

#Perform the over and under sampling on training data
#Print the number of samples in each class after both oversampling and undersampling have been performed
X_train_over, y_train_over = over.fit_resample(X_train, y_train)
print(Counter(y_train_over))
X_train_under , y_train_under = under.fit_resample(X_train_over, y_train_over)
print(Counter(y_train_under))


In [ ]:
#Evaluate initial model accuracy by using 5-Fold Cross-Validation

#Create instance of a Random Forest Classifier model
model = RandomForestClassifier()

#Define a machine learning pipeline with three steps: oversampling, undersampling, and model training
steps = [('over', over), ('under', under), ('model', model)]
pipeline = Pipeline(steps=steps)

#Perform 5-Fold Cross Validation on the pipeline
cv = RepeatedStratifiedKFold(n_splits=5,  random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='recall', cv=cv, n_jobs=-1)

#Calculate mean recall score across all cross-validation scores
np.mean(scores)

In [ ]:
#Train a Random Forest Classifier model on the resampled training dataset
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_under, y_train_under)

#Use the trained model to predict the target variable for the test dataset.
#Test dataset is subset of the original dataset
predictions = rf.predict(X_test)

In [ ]:
#Print classification report
print(classification_report(y_test,predictions))
#Show confusion matrix
ConfusionMatrixDisplay.from_predictions(y_test,predictions)

## **Task 1b** ##

In [ ]:
#Merge change dataframe with incident dataframe
merged_df = change_df.merge(incident_df,how='inner',left_on='number',right_on='caused_by_change')

#For splitting merged_df into training and testing datasets
def sample_first_prows(data, perc=0.9):
    return data.head(int(len(data)*(perc)))

#Split merged_df
merge_train = sample_first_prows(merged_df)
merge_test = merged_df.iloc[max(merge_train.index):]

#New column to store the hour at which the change started
merged_df['start_hour']  = merged_df['opened_at_y'].dt.hour
#Group data frame
#merged_df.groupby('start_hour')[['start_hour']].agg(['count'])

#New dataframes to take subsets of merged_df and merge_train
app_df = merged_df[['number_x','u_applications_impacted.1_map',
                    'u_applications_impacted.2_map','u_applications_impacted.3_map',
                    'u_applications_impacted.4_map','u_applications_impacted.5_map']]
app_df2 = merge_train[['u_application_map','u_applications_impacted.1_map',
                       'u_applications_impacted.2_map','u_applications_impacted.3_map',
                       'u_applications_impacted.4_map','u_applications_impacted.5_map']]


#new_app = pd.melt(app_df, id_vars =['number_x'], value_vars =['u_applications_impacted.1_map','u_applications_impacted.2_map','u_applications_impacted.3_map','u_applications_impacted.4_map','u_applications_impacted.5_map'])
#test_app and app_df2 required for dictionaries
#Melt the u_applications_impacted columns, turning them into rows
test_app = pd.melt(app_df2, 
                   id_vars =['u_application_map'], 
                   value_vars =['u_applications_impacted.1_map','u_applications_impacted.2_map',
                                'u_applications_impacted.3_map','u_applications_impacted.4_map',
                                'u_applications_impacted.5_map'])

#Similar steps for platform dictionary
plat_df = merged_df[['u_platform_map','u_platform_impacted.1_map',
                     'u_platform_impacted.2_map','u_platform_impacted.3_map',
                     'u_platform_impacted.4_map','u_platform_impacted.5_map']]
plat_df = pd.melt(plat_df, 
                  id_vars = ['u_platform_map'], 
                  value_vars=['u_platform_impacted.1_map','u_platform_impacted.2_map',
                              'u_platform_impacted.3_map','u_platform_impacted.4_map',
                              'u_platform_impacted.5_map'])
plat_df = plat_df.drop_duplicates()
plat_df = plat_df[(plat_df['value'] != 'Unknown or Not Applicable') 
                  & (plat_df['u_platform_map'] != 'Unknown or Not Applicable') ]

In [ ]:
#Create dictionaries to store weights

#Dictionary 1 -- Application Dictionary to Map Applications
test_app = test_app.drop_duplicates()
test_app = test_app[test_app['value'] != 'Unknown or Not Applicable']
d = defaultdict(set)
for counter, x in enumerate(test_app.u_application_map):
    y = test_app.iloc[counter,2]
    if x not in d.keys():
        d[x].add(y)
    if y not in d[x]:
        d[x].add(y)

#Dictionary 2 -- Platform Dictionary to Map Platforms
plat_dict = defaultdict(set)
for counter, x in enumerate(plat_df.u_platform_map):
    y = plat_df.iloc[counter,2]
    if x not in plat_dict.keys():
        plat_dict[x].add(y)
    if y not in d[x]:
        plat_dict[x].add(y)

In [ ]:
#Define variable to store merged dataframe
df = merged_df
#Define empty list to store "hits"
hits = []
#Define empty list to store dataframe lengths
df_lens= [] 

#Take subset of columns to use in identifying possible changes
test_inc = df[['opened_at_y','u_application_map','u_business_map_y','u_platform_map','caused_by_change']]

#Convert dataframe to list of flat lists
#Loop over each row of merged dataframe 
for x in range(len(df)):
    #Select single row at each iteration and store in test_inc2
    test_inc2 = test_inc[x:x+1]
    flat_list = []
    #Convert values of dataframe to list
    test_list = test_inc2.values.tolist()
    #Flatten list and store in flat_list
    for sublist in test_list:
        for item in sublist:
            flat_list.append(item)

#Look for change 2 days before incident and 3 days after
refined_df = df[((df['closed_at_x']>=flat_list[0] - timedelta(days=3)) 
                      & (df['closed_at_x']<=flat_list[0] + timedelta(days=2)))] 

#Take subset of refined_df
refined_df = refined_df[['u_applications_impacted.1_map','u_applications_impacted.2_map',
                         'u_applications_impacted.3_map','u_applications_impacted.4_map',
                         'u_applications_impacted.5_map', 'u_platform_impacted.1_map',
                         'u_platform_impacted.2_map','u_platform_impacted.3_map',
                         'u_platform_impacted.4_map','u_platform_impacted.5_map',
                         'u_business_map_x', 'opened_at_y','caused_by_change']]


In [ ]:
#Add a weight and a flag column for each column of refined_df
#These will store the weights for each category and whether or not it was flagged
for x in range(11):
    refined_df['weight' + str(x + 1)] = .10
    refined_df['flag' + str(x+1) ] = 0

In [ ]:
#See first five rows of refined_df
refined_df.head(5)

In [ ]:
#Store the ICE applications impacted by the change
app_columns = ['u_applications_impacted.1_map','u_applications_impacted.2_map',
               'u_applications_impacted.3_map',	'u_applications_impacted.4_map',
               'u_applications_impacted.5_map']

#The index of the first weight column for the category that corresponds to applications impacted
col_index = 14

#Update weight columns for appropriate category if not unknown or inapplicable
if flat_list[1] != 'Unknown or Not Applicable':
    for x in app_columns:
        #Check whether the application impacted is present in a dictionary called d
        if flat_list[1] not in d.keys():
            break
        #Check whether the value in the current column (x) is present in the list of impacted applications (d[flat_list[1]])
        for count, y in enumerate(refined_df[x]):
            #If present, set the corresponding weight column for the category to 1.
            if  y in d[flat_list[1]]:
                refined_df.iloc[count,col_index] =1
            if count == len(refined_df) -1:
                col_index = col_index + 2

In [ ]:
#Store the ICE platforms impacted by the change
plat_columns = ['u_platform_impacted.1_map', 'u_platform_impacted.2_map',
                'u_platform_impacted.3_map','u_platform_impacted.4_map',
                'u_platform_impacted.5_map']

#The index of the first weight column for the category that corresponds to platforms impacted
col_index = 24

#Update weight columns for appropriate category if not unknown or inapplicable
if flat_list[3] != 'Unknown or Not Applicable':
    for x in plat_columns:
        if flat_list[3] not in d.keys():
            break
        for count, y in enumerate(refined_df[x]):
            if  y in d[flat_list[3]]: #Should this be plat_dict?
                refined_df.iloc[count,col_index] =1
            if count == len(refined_df) -1:
                col_index = col_index + 2

In [ ]:
#Update weight columns for impacted businesses

#If third element is unknown or not applicable, then set the variable col_index to 34
if flat_list[2] != 'Unknown or Not Applicable':
    col_index =34
    #Iterate over the values in the 'u_business_map_x' column of the DataFrame refined_df
    for count, x in enumerate(refined_df.u_business_map_x):
        #Check whether each value is equal to third element in flat_list
        if x == flat_list[2]:
            #Sets the value of the corresponding row and column in refined_df to 1.
            refined_df.iloc[count,col_index] =1 

In [ ]:
#Gather weights and flags in new dataframes
weights = refined_df[['weight1','weight2','weight3','weight4',
                      'weight5','weight6','weight7','weight8',
                      'weight9','weight10','weight11']]
flags = refined_df[['flag1','flag2','flag3','flag4',
                    'flag5','flag6','flag7','flag8',
                    'flag9','flag10','flag11']]

#Convert to numpy array
np_weights = weights.to_numpy()
#print(np_weights)
np_flag = flags.to_numpy()
#print(np_flag)

#Calculate probabilities based on weights and flag
prob_score = np.sum(np_weights*np_flag,axis=1)

#Store probability scores in a new column
refined_df['probability'] = prob_score

#Rank each probability score
refined_df['rank'] = refined_df['probability'].rank(ascending=False)

#Show top five
refined_df = refined_df[refined_df['rank']<=5]

In [ ]:
#Record if any hits
if flat_list[4] in refined_df['caused_by_change'].values:
    hits.append(1)
else:
    hits.append(0) 

#Append length of refined_df
df_lens.append(len(refined_df)) 

In [ ]:
df_lens
sum(hits)
refined_df.head(15)

## PART 5: *Machine Learning for Task 2*

# UNDER CONSTRUCTION #

In [ ]:
#Set binary response variable
inc_ml.loc[:,'response'] = inc_ml.caused_by_change.map(lambda x: 0 if x =='Unknown or Not Applicable' else 1)

inc_ml.head(5)

In [ ]:
#Perform oversampling to handle class imbalancing in the dataset
X = inc_ml.iloc[:,0:12]
y = inc_ml.iloc[:,-1]

smote = SMOTE()


label_encoder = LabelEncoder()
for i in X.columns:
  X[i] = label_encoder.fit_transform(X[i])


print(Counter(y))
X = X.values
y = y.values
X, y = smote.fit_resample(X,y)

print(Counter(y))

chi2_selector = SelectKBest(chi2, k= 'all')
X_kbest = chi2_selector.fit_transform(X, y)

X = X_kbest

In [ ]:
#Define classifiers
log_reg_model = LogisticRegression(random_state = 42)
decision_tree_model = DecisionTreeClassifier(criterion="entropy",
                                     random_state=0)
knn = KNeighborsClassifier()
rfc = RandomForestClassifier(random_state = 42)
#accuracies = cross_val_score(knn, X_train, y_train, cv=5)
cross_validation(rfc,X,y)
#log_result = cross_validation(log_reg_model, X, y, 5)
#print(log_result)

In [ ]:
#Split data into training and testing data sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=42, shuffle=True)
#decision_tree_model.fit(X_train,y_train)
#y_pred = decision_tree_model.predict(X_test)

#decision_tree_model.score(X_test,y_test)

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)


#log_reg_model.fit(X_train,y_train)
#lg_pred = log_reg_model.predict(X_test)
#log_reg_model.score(X_test, y_test)

#knn.fit(X_train,y_train)
#y_pred = knn.predict(X_test)
#knn.score(X_test,y_test)

#Fit random forest on training data
rfc.fit(X_train,y_train)
#Make predictions on testing data
y_pred = rfc.predict(X_test)


In [ ]:
#Visually compare the actual results and the predicted results
data = {"y_true":y_test,"y_prediction":y_pred}

test = pd.DataFrame(data)

test.head(5)

# SANDBOX BELOW - WILL NOT BE INCLUDED IN FINAL #

In [ ]:
#Use a Confusion Matrix to display the results
ConfusionMatrixDisplay.from_predictions(y_test,y_pred)

In [ ]:
#time_diff_df = merged_df[['closed_at_x','opened_at_y']]
#time_diff_df['time_diff'] = time_diff_df.closed_at_x - time_diff_df.opened_at_y

test_df = merged_df[:10]
test_df

In [ ]:
test_inc = test_df[['opened_at_y','u_application_map','u_business_map_y']]
test_inc = test_inc[5:6]
test_list = test_inc.values.tolist()
flat_list = []
for sublist in test_list:
    for item in sublist:
        flat_list.append(item)

refined_df = test_df[((test_df['closed_at_x']>=flat_list[0] - timedelta(days=60)) & (test_df['closed_at_x']<=flat_list[0] + timedelta(days=2)))]

refined_df = refined_df[['u_applications_impacted.1_map','u_applications_impacted.2_map','u_applications_impacted.3_map','u_applications_impacted.4_map','u_applications_impacted.5_map', 'u_business_map_x', 'opened_at_y','caused_by_change']]

In [ ]:


insert = 1
for x in range(6):
    if x>0:
        insert +=2
    refined_df.insert(insert,'app_weight' + str(x+1),.15)


In [ ]:
import numpy as np

np1 = np.array([[.15,.15,.15,.15,.15,.15],[.15,.15,.15,.15,.15,.15]])

np2 = np.array([[0,1,0,0,1,0],[0,1,0,0,1,0]])

np.sum(np1 * np2, axis = 1)

In [ ]:
refined_df.insert(17, 'app6_flag',refined_df['u_applications_impacted.1_map'].map(lambda x: 1 if x == flat_list[1] else 0))
refined_df

In [ ]:
weights = refined_df[['app_weight1','app_weight2','app_weight3','app_weight4','app_weight5']]

flags = refined_df[['app1_flag','app2_flag','app3_flag','app4_flag','app5_flag']]

np_weights = weights.to_numpy()

np_flag = flags.to_numpy()

np.sum(np_weights*np_flag,axis=1)